In [13]:
import helics as h
from opendssdirect import dss
import pandas as pd
import time
import threading  # To run the broker in parallel
import json

base_dir = r"C:/Users/nicol/Helics_experimental"

import os
os.chdir(base_dir)

In [14]:
# Function to start a HELICS broker
def start_broker():
    global broker
    broker = h.helicsCreateBroker("zmq", "", "--federates=1")

# Start the broker in a separate thread
broker_thread = threading.Thread(target=start_broker, daemon=True)
broker_thread.start()
time.sleep(1)  # Allow the broker to initialize

In [15]:
# Function to run the OpenDSS federate
def run_opendss_federate():
    fedinfo = h.helicsCreateFederateInfo()
    h.helicsFederateInfoSetCoreName(fedinfo, "OpenDSS_Federate")
    h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")  # Connect to the broker
    #h.helicsFederateInfoSetBroker(fedinfo, "tcp://localhost:23404")
    h.helicsFederateInfoSetTimeProperty(fedinfo, h.HELICS_PROPERTY_TIME_DELTA, 1.0)

    # Create HELICS federate
    fed = h.helicsCreateValueFederate("OpenDSS_Federate", fedinfo)

    # Register HELICS publication (voltage)
    pub = h.helicsFederateRegisterPublication(fed, "voltage_out", h.HELICS_DATA_TYPE_STRING, "")

    # Enter execution mode
    h.helicsFederateEnterExecutingMode(fed)
    
    #run one timestep of OpenDSS and publish a voltage value
    #Identify the subdirectory and file names
    data_dir = base_dir + "/data/"
    feeder_file_path = data_dir + 'ieee37.dss'

    dss.Command('Redirect ' + feeder_file_path)
    
    time_step = 0
    while time_step < 10:  # Simulate 10 time steps
        
        dss.Solution.Solve()

        V = dss.Circuit.AllBusMagPu()

        # Publish voltage data to HELICS
        # Get voltage magnitudes per bus
        bus_names = dss.Circuit.AllBusNames()
        voltages = dss.Circuit.AllBusMagPu()

        voltage_dict = {}
        for i, bus in enumerate(bus_names):
            voltage_dict[bus] = voltages[i]

        # Publish the entire voltage dict as JSON
        voltage_json = json.dumps(voltage_dict)
        h.helicsPublicationPublishString(pub, voltage_json)
        print(f"Published voltages: {voltage_json}")
    
        # Request the next HELICS time step
        granted_time = h.helicsFederateRequestTime(fed, time_step + 1)
        time_step = granted_time


    h.helicsFederateFinalize(fed)
    print("[OpenDSS Federate] Finalized.")


In [16]:
solar_data = pd.read_csv(r"C:\Users\nicol\Helics_experimental\data\solar_data.csv")
solar_data.columns = solar_data.columns.str.replace('_pv$', '', regex=True)
# Create a 'time' column from the DataFrame's index (each row represents a time step)
solar_data['time'] = solar_data.index

load_data = pd.read_csv(r"C:\Users\nicol\Helics_experimental\data\load_data.csv")
load_data['time'] = load_data.index
load_data.sort_values('time', inplace=True)

# ---------------------------
# Determine the list of node names (all columns except 'time')
# ---------------------------
node_names = [col for col in solar_data.columns if col != 'time']

def get_values_at_time(t, df):
    """
    Returns a dictionary of {node: value} for the given time t from DataFrame df.
    If t is not present, returns the last available row.
    """
    if t in df['time'].values:
        row = df[df['time'] == t].iloc[0]
    else:
        row = df.iloc[-1]
    # Return all node values (exclude 'time')
    return row.drop('time').to_dict()


In [17]:
def run_voltage_consumer_federate():
    fedinfo = h.helicsCreateFederateInfo()
    h.helicsFederateInfoSetCoreName(fedinfo, "Voltage_Consumer_Federate")
    h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")
    h.helicsFederateInfoSetTimeProperty(fedinfo, h.HELICS_PROPERTY_TIME_DELTA, 1.0)

    # Create the HELICS federate
    fed = h.helicsCreateValueFederate("Voltage_Consumer_Federate", fedinfo)

    # Register publication (net demand output) and subscription (voltage input)
    pub = h.helicsFederateRegisterPublication(fed, "voltage", h.HELICS_DATA_TYPE_STRING, "")
    sub = h.helicsFederateRegisterSubscription(fed, "voltage_out", "")

    # Enter execution mode
    h.helicsFederateEnterExecutingMode(fed)
    
    current_time = 0
    end_time = 10
    time_step = 1

    voltage_timeseries = []  # 🔥 NEW: store voltage per timestep

    while current_time < end_time:
        solar_values = get_values_at_time(current_time, solar_data)
        load_values = get_values_at_time(current_time, load_data)

        net_demand = {}
        for node in node_names:
            load_val = load_values.get(node, 0)
            pv_val = solar_values.get(node, 0)
            net_demand[node] = load_val - pv_val

        print(f"Time: {current_time} | Net Demand: {net_demand}")

        net_demand_json = json.dumps(net_demand)
        h.helicsPublicationPublishString(pub, net_demand_json)

        # Advance time
        next_time = current_time + time_step
        current_time = h.helicsFederateRequestTime(fed, next_time)

        # ✅ NEW: Get voltage data (JSON string) from OpenDSS federate
        try:
            voltage_json = h.helicsInputGetString(sub)
            voltage_data = json.loads(voltage_json)
            voltage_data['time'] = current_time
            voltage_timeseries.append(voltage_data.copy())
            print(f"Time: {current_time} | Voltages: {voltage_data}")
        except Exception as e:
            print(f"[ERROR] Failed to receive voltage data: {e}")

    # Finalize federate
    h.helicsFederateFinalize(fed)
    print("[Voltage Consumer Federate] Finalized.")

    # ✅ NEW: Save voltage data for plotting
    try:
        voltage_df = pd.DataFrame(voltage_timeseries)
        voltage_df.to_csv("voltage_timeseries.csv", index=False)
        print("[Voltage Data] Saved to 'voltage_timeseries.csv'")
    except Exception as e:
        print(f"[ERROR] Could not save voltage data: {e}")


In [18]:
#Run the federates and then finalize/close
# Start both federates in separate threads
opendss_thread = threading.Thread(target=run_opendss_federate)
consumer_thread = threading.Thread(target=run_voltage_consumer_federate)

opendss_thread.start()
consumer_thread.start()

# Wait for both federates to finish
opendss_thread.join()
consumer_thread.join()

# Close the HELICS broker
if 'broker' in globals() and h.helicsBrokerIsConnected(broker):
    h.helicsBrokerDisconnect(broker)
    h.helicsBrokerFree(broker)

print("Simulation complete. Broker closed.")

Time: 0 | Net Demand: {'S701a': 48.65106259999999, 'S701b': 17.491247200000004, 'S701c': 99.20200310000001, 'S712c': 1.7962140900000065, 'S713c': 28.902324879999995, 'S714a': 1.6562613200000005, 'S714b': 1.7558022599999994, 'S718a': 14.24384101999999, 'S720c': 30.256853229999997, 'S722b': 25.700220239999993, 'S722c': 6.018016620000001, 'S724b': 1.665514139999999, 'S725b': 12.47717376, 'S727c': 13.915412909999997, 'S728': 28.544458140000003, 'S728a': -24.46035039, 'S729a': 12.648238430000003, 'S730c': 28.344410949999997, 'S731b': 3.1242751999999996, 'S732c': 7.800037530000004, 'S733a': 10.032473209999992, 'S734c': 5.126738349999997, 'S735c': 1.700992130000003, 'S736b': 2.9369750199999984, 'S737a': 19.734600799999996, 'S738a': 17.49677460000001, 'S740c': 5.878095540000004, 'S741c': 11.614285939999998, 'S742a': 1.6690445349999994, 'S742b': 23.17490557, 'S744a': 6.950545559999998}
[ERROR] Failed to receive voltage data: 'float' object does not support item assignment
Time: 1.0 | Net Demand

c:\Users\nicol\anaconda3\envs\magic_exp_PG\lib\site-packages\helics\capi.py:4444: UserWarning: This function is deprecated. Use `helicsFederateDisconnect` instead.
  warnings.warn("This function is deprecated. Use `helicsFederateDisconnect` instead.")
